# 2: The Fermi Tools

To do a full analysis, we need to do lots of mathsy things to our data files, and while we can do this ourselves, it would involve a lot of (wo)manpower and patience. Thats why NASA developed the Fermi Science Tools, to make this all a lot easier!

We will be looking at executing a binned analysis, where we bin the data in space and energy. There are also unbinned analyses, although these are less commonly used for our purposes.

<div style='border:DimGray; background:#c9af98; color:#3a4660; font-size:12pt'>
<ol>
    <li>Use <strong><em>gtselect</em></strong> to select the photons we actually want from our downloaded photon files.</li>
    <br>
    <li>Use <strong><em>gtmktime</em></strong> to make time cuts to the data based on the position of the satellite (for example to avoid times when the spacecraft was making orbital adjustments or flying over the South Atlantic Anomaly).</li>
    <br>
    <li>Use <strong><em>gtbin</em></strong> to bin the data in space and in energy.</li>
<br>
    <li>Make a model of the sky seeded with sources from the 4FGL (the fourth Fermi-LAT point source catalog) with the tool <strong><em>make4FGLxml.py</em></strong>.</li>
    <br>
    <li>Calculate the livetimes using <strong><em>gtltcube</em></strong>. Livetime is how long the LAT observed a particular point of the sky at a particular angle.</li>
    <br>
    <li>Calculate exposure using <strong><em>gtexpcube2</em></strong>. Exposure is equal to livetime multiplied by the effective area of the instrument. </li>
    <br>
    <li>Calculate the source maps using <strong><em>gtsrcmaps</em></strong>. The sourcemap is applying our exposure to each source in the model.</li>
    <br>
    <li>Do a likelihood fit to the data.</li>
</ol>
</div>

<!-- 
1. Use *gtselect* to select the photons we actually want from our downloaded photon files.
2. Use *gtmktime* to make time cuts to the data based on the position of the satellite (for example to avoid times when the spacecraft was making orbital adjustments or flying over the South Atlantic Anomaly).
3. Use *gtbin* to bin the data in space and in energy.
4. Make a model of the sky seeded with sources from the 4FGL (the fourth Fermi-LAT point source catalog) with the tool *make4FGLxml.py*.
5. Calculate the livetimes using *gtltcube*. Livetime is how long the LAT observed a particular point of the sky at a particular angle
6. Calculate exposure using *gtexpcube2*. Exposure is equal to livetime multiplied by the effective area of the instrument. 
7. Calculate the source maps using *gtsrcmaps*. The sourcemap is applying our exposure to each source in the model.
8. Do a likelihood fit to the data. -->

FYI, the South Atlantic Anomaly is the point where the Van Allen radiation belts are their closest to Earth. The LAT shuts off whilst its passing through the anomaly, due to the intense radiation on the detector. 

Firstly lets look at gtselect (gt stands for GLAST tools, and prefixes all the Fermi tool names).

## gtselect

This tool is used to filter the data by region of interest (ROI), time and some other things called event classes and event types.

Event classes and event types are determined by how likely a given event was to be a photon, and how good the reconstruction of the event was. For each combination of these classes and types, there is a different instrument response function (IRF) to go with it - this tells the software what effective area and point spread function (PSF) to use when doing a likelihood fit or working out the source map (which we shall come to!).

### Event Classes

There are several event classes in the Fermi-LAT data. These are often referred to as TRANSIENT, SOURCE, CLEAN, ULTRACLEAN and ULTRACLEANVETO in order of increasingly stringent cuts. There are several more weakly cut data sets for more specific analyses (Solar physics etc.), but these are the main ones that you will work with. In the Fermi tools, each of these classes is referred to in a bitmask, but when making selections you simply refer to the particular power of 2 corresponding to your class. 

* TRANSIENT, class = 64
* SOURCE, class = 128
* CLEAN, class = 256
* ULTRACLEAN, class = 512
* ULTRACLEANVETO, class = 1024

Typically we use SOURCE for most point source analyses, although we may use more stringent requirements like CLEAN, for analysis on the Galactic plane, or of the Galactic diffuse emission.

### Event Types

The event types instead determine how you want to analyse the data. This allows you several choices. The full list of available options are:
+ FRONT (1)
+ BACK (2)
+ PSF0 (4)
+ PSF1 (8)
+ PSF2 (16)
+ PSF3 (32)
+ EDISP0 (64)
+ EDISP1 (128)
+ EDISP2 (256)
+ EDISP3 (512)

A standard, simple analysis would use both FRONT and BACK converting events, thus one would use:

evtype = FRONT+BACK = 1+2 = 3

The PSF and EDISP classes allow you to split the data into quartiles according to the best PSF reconstruction (PSF3) or the smallest energy dispersion (EDISP3). Occasionally it is very useful to improve your PSF (for example if you're looking for faint sources, those on the galactic plane, or in some cases DM Halos.

### Using the tool

In most cases (including ours), the Fermi-LAT data server will give you several photon files (PH00, PH01, PH02...) that you can combine with *gtselect* (or using the HEASOFT fits utility *ftmerge* but we won't cover that in this tutorial). To do this, you first make a list of these files. In this case:


In [2]:
!ls data/*PH*.fits > events.txt

In [4]:
import os
if(not os.path.exists("/workdir/NGC_1275_Fit")):
    os.mkdir("/workdir/NGC_1275_Fit")
    os.listdir("/workdir/NGC_1275_Fit")
else:
    os.listdir("/workdir/NGC_1275_Fit")

We can call *gtselect* into Python or on the command line. Here, we will do it from the command line (just type 'gtselect'). We will then be presented with a number of options, which we will need to fill in. 

For example:
```CLI
$/> gtselect evclass=128 evtype=3

    Input FT1 file[fermievents1.txt] events.txt 

    Output FT1 file[V404_filtered.fits] NGC_1275_filtered.fits

    RA for new search center (degrees) (0:360) [306.01602884] 49.9507

    Dec for new search center (degrees) (-90:90) [33.86717685] 41.5117

    radius of new search region (degrees) (0:180) [20] 15

    start time (MET in s) (0:) [247190401] 283996802

    end time (MET in s) (0:) [562723205] 315532802

    lower energy limit (MeV) (0:) [100] 100

    upper energy limit (MeV) (0:) [300000] 300000
    
    maximum zenith angle value (degrees) (0:180) [90] 90
```

Ignore what is in the square brackets, those are simply your choices from the last time you used gtselect. You can see from our prevoius output file name that we were recently looking at the microquasar V404 Cygni over a 10 year period.

Most of these choices come from our data query. Note that we input our event class and event type on the first line, with the call for *gtselect*. The zenith angle (simply put) is our angle from the pointing of the spacecraft at which it accepts photons, we use 90 degrees because this cuts out photons from the Earth (the Earth's atmosphere is a large source of $\gamma$-rays, which we don't wish to observe!) 90 degrees is a standard cut, but you can be less stringent depending on what you want to do, for more advice see Fermi's article on [data selection and preparation](http://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Data_Exploration/Data_preparation.html).

Now we have a trimmed (or filtered) photon file that only includes the event types and classes we want. 

## gtmktime

The *gtmktime* tool takes a photon file and applies "good time intervals" to the data, which are defined by time when the spacecraft was not in data taking mode, flying over the South Atlantic Anomaly, even sometimes in which a particularly bright GRB has gone off (messing up analyses), etc.

We shall use a filter expression "(DATA_QUAL>0)&&(LAT_CONFIG==1)" which basically says "the LAT is working as intended". For transient and burst analysis this can be relaxed, and the link above also gives a list of these filter expressions to work with. We shall not use a ROI-based zenith angle cut as this is not recommended with the Pass 8 data.  

For example:
```CLI
$/> gtmktime

    Spacecraft data file[lat_spacecraft.fits] data/L1910170847288E15477E17_SC00.fits

    Filter expression[(DATA_QUAL>0)&&(LAT_CONFIG==1)] 

    Apply ROI-based zenith angle cut[no] 

    Event data file[V404_filtered.fits] NGC_1275_filtered.fits
    
    Output event file name[V404_lt_filtered.fits] NGC_1275_gti.fits
```


Now we have our photons filtered by time, as well as photon quality!

## gtbin

Now we'll bin the data so we can make some nice pictures of the data. The *gtbin* tool takes care of this for you, but first we have to do some maths. Our ROI is cut so that we have all events within 15 degrees of the source, but we usually want to cut this down to a square that will fit inside this circle so we don't have big black edges around it. 

We want to work out the size of the red square region that fits in the ROI:

<img src="img/Rect.png" width="300" height="300" />

We know the diagonal is 15 degrees long, so the width and height are approximately 15sqrt(2). Separating this into 0.1x0.1 degree bins, we can then run *gtbin*:

```CLI
$/> gtbin
    
    This is gtbin version ScienceTools-v11r5p3-fssc-20170716
    
    Type of output file (CCUBE|CMAP|LC|PHA1|PHA2|HEALPIX) [CMAP] CCUBE
    
    Event data file name[picA_gti.fits] NGC_1275_gti.fits
    
    Output file name[picA_cmamp_small.fits] NGC_1275_ccube.fits
    
    Spacecraft data file name[NONE] 
    
    Size of the X axis in pixels[100] 212
    
    Size of the Y axis in pixels[100] 212
    
    Image scale (in degrees/pixel)[0.2] 0.1
    
    Coordinate system (CEL - celestial, GAL -galactic) (CEL|GAL) [CEL] CEL
    
    First coordinate of image center in degrees (RA or galactic l)[79.9571] 49.9507
    
    Second coordinate of image center in degrees (DEC or galactic b)[-45.46] 41.5117
    
    Rotation angle of image axis, in degrees[0] 0
    
    Projection method e.g. AIT|ARC|CAR|GLS|MER|NCP|SIN|STG|TAN:[AIT] AIT
    
    Algorithm for defining energy bins (FILE|LIN|LOG) [LOG] LOG
    
    Start value for first energy bin in MeV[100] 100
    
    Stop value for last energy bin in MeV[10000] 300000
    
    Number of logarithmically uniform energy bins[20] 30
    
    gtbin: WARNING: No spacecraft file: EXPOSURE keyword will be set equal to ontime.
```

<div class="alert alert-block alert-info"> <b>Tip:</b> With this tool, you can also make a counts map (CMAP), which is not binned in energy. </div>

## make4FGLxml.py

Before we go any further and start pre-computing some values for our eventual likelihood fitting, we will make an xml file of sources near to our ROI. We need to take account of sources outside of our ROI (up to 10 degrees away) to make sure that we don't miss anything out. There's a great script called "make4FGLxml.py" that does this for you, given some information.

In [5]:
import make4FGLxml as makexml

srcmaker = makexml.srcList('gll_psc_v27.fit', 'NGC_1275_gti.fits', 'NGC_1275_model.xml')
# Give it the catalog, ev file and output filename.
srcmaker.makeModel()


This is make4FGLxml version 01r04.
The default diffuse model files and names are for pass 8 and 4FGL and assume you have v11r5p3 of the Fermi Science Tools or higher.
Creating file and adding sources from 4FGL
Extended source G150.3+4.5 in ROI with %s spatial model.
Extended source HB 9 in ROI, make sure $(FERMI_DIR)/data/pyBurstAnalysisGUI/templates/HB9.fits is the correct path to the extended template.
Extended source HB 3 in ROI with %s spatial model.
Extended source W 3 in ROI, make sure $(FERMI_DIR)/data/pyBurstAnalysisGUI/templates/W3.fits is the correct path to the extended template.
Added 216 point sources and 4 extended sources
If using unbinned likelihood you will need to rerun gtdiffrsp for the extended sources or rerun the makeModel function with optional argument psForce=True


This tool will include sources in your ROI and -- by default, though this can be changed -- 10 degrees outside your ROI (because the PSF of Fermi at low energies is so dire - this is necessary if you're near a bright source). It also includes your isotropic and diffuse files (by default, if they're in your current directory - hence we have provided them).

This program can also be run from the command line with the input:

```CLI
$/> python make4FGLxml.py gll_psc_v20.fit evfile.fits out_model.xml
```

## gtltcube

Next we're going to calculate the livetime. This tool uses the spacecraft file to add up the inclinations of the LAT over the mission lifetime in a given direction. There is a really thorough discussion of [livetime and exposure](http://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Data_Exploration/livetime_and_exposure.html) on the Fermi wesbite if you fancy a read (it's probably worth it). The reason we do this is because the instrument responds differently to a photon if it hits the detector on-axis or off-axis (which hopefully makes sense!). We use this information to then calculate the exposure.

The most important thing (or most annoying thing) to remember about ltcubes is that they take ####ing forever to make. This is what takes a *really* long time if you're making lightcurves as well.

Usually it's easiest to make an ltcube for the whole sky, since it doesn't really take much longer than for just your ROI (it has to iterate through the whole spacecraft history anyway). Here is what running gtltcube looks like:

```CLI
$/> gtltcube zmax=90

    WARNING: version mismatch between CFITSIO header (v3.43) and linked library (v3.41).

    Event data file[NGC_1275_gti.fits] 

    Spacecraft data file[data/L1910170847288E15477E17_SC00.fits] 

    Output file[ltcube.fits]

    Step size in cos(theta) (0.:1.) [0.025]

    Pixel size (degrees)[1] 

    Working on file data/L1910170847288E15477E17_SC00.fits
    
    .....................!
```

aaand we're done!

<div class="alert alert-block alert-info"> <b>Tip:</b> You can reuse any whole-sky ltcube as long as it covers the full time period of your observations. </div>


## gtexpmap2

Welcome to one of the most confusingly named tools in the world. In the Fermi tools there are currently tools named "gtexpmap", "gtexpcube", "gtexpcube2" and "gtexposure". Even the experts often struggle to remember which tool does what! Either way - the one you want for binned analyses is "gtexpcube2" (DO NOT USE GTEXPCUBE - it's old and broken...).

In this tool, we take the ltcube we've just made and run it through gtexpcube2. By the way, the output from this tools is often referred to as a "bexpmap", and the ltcube is referred to as the "expmap". This has made a lot of people angry and been widely regarded as a stupid idea.

What this tool actually does, is compute the expoure of our instrument over our ROI, not all areas of the sky recieve even coverage, so this tool accounts for that. As a useful visual aid, this is an (old) example of what the exposure of the instrument looks like. 

<img src="img/exposure.png" width="500" height="300" />

Just because we have data for the whole sky, doesn't mean we have an even coverage! We can see almost a factor of 2 difference in the exposure here, definitely enough to seriously affect our results. Lets run our exposure map then, and see our inputs.

```CLI
$/> gtexpcube2

    WARNING: version mismatch between CFITSIO header (v3.43) and linked library (v3.41).

    Livetime cube file[] ltcube.fits

    Counts map file[] none

    Output file name[] NGC_1275_bexpmap.fits

    Response functions to use[CALDB] P8R3_SOURCE_V3

    Size of the X axis in pixels[INDEF] 212

    Size of the Y axis in pixels[INDEF] 212

    Image scale (in degrees/pixel)[INDEF] .1

    First coordinate of image center in degrees (RA or galactic l)[INDEF] 49.9507

    Second coordinate of image center in degrees (DEC or galactic b)[INDEF] 41.5117

    Rotation angle of image axis, in degrees[0.] 0

    Projection method e.g. AIT|ARC|CAR|GLS|MER|NCP|SIN|STG|TAN[CAR] AIT

    Coordinate system (CEL - celestial, GAL -galactic) (CEL|GAL) [GAL] CEL

    Start energy (MeV) of first bin[INDEF] 100

    Stop energy (MeV) of last bin[INDEF] 300000

    Number of logarithmically-spaced energy bins[INDEF] 30

    Computing binned exposure map.....................!
```


Done! Now we have computed the exposure, we can move on to generating a source map of our ROI.

## gtsrcmaps

This tool is how we manage to do likelihood fitting to the data within a reasonable timeframe. This tool generates a source map, combining our model .xml file, with our exposure map, instrument response function and cut down data to essentially give us our 'final' model file, with which we can do our advanced analysis, and actually get some science out! Here's how it works:

```CLI
$/> gtsrcmaps

    WARNING: version mismatch between CFITSIO header (v3.43) and linked library (v3.41).

    Exposure hypercube file[ltcube.fits] 

    Counts map file[NGC_1275_ccube.fits] 

    Source model file[NGC_1275_model.xml] 

    Binned exposure map[NGC_1275_bexpmap.fits] 

    Source maps output file[NGC_1275_srcmaps.fits] 

    Response functions[P8R3_SOURCE_V3]

    Done!
```

And there we go! We haven't included these files in the tutorial package (they're easy enough to generate yourself, following these steps). But in principle, we are now ready to go and do some science with our model of NGC 1275 and its surrounding sky region.

In the next tutorial, we'll be looking at using *fermipy*, which will make all of this, much easier. With *fermipy* we will go through these steps again, but it will also allow us to do some more advanced analysis too.